In [14]:
# XGBOOST model 

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from colorama import Fore, Style

# USING FUNCTION TO HANDLE OUTLIERS
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# FUNCTION VOLATILITY BANDS
def calculate_volatility_bands(df, column, window=20, factor=2):
    rolling_std = df[column].rolling(window=window).std()
    upper_band = df[column] + factor * rolling_std
    lower_band = df[column] - factor * rolling_std
    return upper_band, lower_band

# DATASET
df_csv = pd.read_csv(r"C:\Users\nh013\Desktop\Crypto fluctuation of 3 consecutive years\Cryptocurrency Prices by Date.csv")
df_xlsx = pd.read_excel(r"C:\Users\nh013\Desktop\Crypto fluctuation of 3 consecutive years\All Currencies Table.xlsx")

# SELECT COLUMNS
df_csv = df_csv[['Currency', 'Date', 'Price']]
df_xlsx = df_xlsx[['Price', 'Name', 'Market Cap', 'Circulating Supply', 'Volume (24hr)']]

# IDENTIFY MISSING VALUES
df_csv.dropna(inplace=True)
df_xlsx.dropna(inplace=True)

# CONVERT 'Price' COLUMN TO NUMERIC
df_csv['Price'] = pd.to_numeric(df_csv['Price'], errors='coerce')
df_xlsx['Price'] = pd.to_numeric(df_xlsx['Price'], errors='coerce')

# FILL MISSING VALUES WITH MEAN
mean = df_csv['Price'].mean()
df_csv.fillna(mean, inplace=True)
mean = df_xlsx['Price'].mean()
df_xlsx.fillna(mean, inplace=True)

# HANDLE OUTLIERS
df_csv = handle_outliers(df_csv, 'Price')
df_xlsx = handle_outliers(df_xlsx, 'Price')

# NORMALIZE AND SCALE
scaler_csv = MinMaxScaler()
num_cols_csv = df_csv.select_dtypes(include='number').columns
df_csv[num_cols_csv] = scaler_csv.fit_transform(df_csv[num_cols_csv])

scaler_xlsx = MinMaxScaler()
num_cols_xlsx = df_xlsx.select_dtypes(include='number').columns
df_xlsx[num_cols_xlsx] = scaler_xlsx.fit_transform(df_xlsx[num_cols_xlsx])

# LABEL ENCODING FOR 'Name' COLUMN
label_encoder = LabelEncoder()
df_xlsx['Name'] = label_encoder.fit_transform(df_xlsx['Name'])

# CALCULATE VOLATILITY BANDS
upper_band_csv, lower_band_csv = calculate_volatility_bands(df_csv, 'Price', window=20, factor=2)
df_csv['Upper Band'] = upper_band_csv.fillna(upper_band_csv.mean())
df_csv['Lower Band'] = lower_band_csv.fillna(lower_band_csv.mean())

upper_band_xlsx, lower_band_xlsx = calculate_volatility_bands(df_xlsx, 'Price', window=20, factor=2)
df_xlsx['Upper Band'] = upper_band_xlsx.fillna(upper_band_xlsx.mean())
df_xlsx['Lower Band'] = lower_band_xlsx.fillna(lower_band_xlsx.mean())

# PREPARE DATA FOR XGBOOST
X_csv = df_csv.drop(['Currency', 'Date'], axis=1)
y_csv = df_csv['Price']

X_xlsx = df_xlsx.drop('Price', axis=1)
y_xlsx = df_xlsx['Price']

# TRAIN-TEST SPLIT
X_train_csv, X_test_csv, y_train_csv, y_test_csv = train_test_split(X_csv, y_csv, test_size=0.2, random_state=42)
X_train_xlsx, X_test_xlsx, y_train_xlsx, y_test_xlsx = train_test_split(X_xlsx, y_xlsx, test_size=0.2, random_state=42)

# XGBOOST MODEL
xgb_model_csv = xgb.XGBRegressor()
xgb_model_csv.fit(X_train_csv, y_train_csv)

xgb_model_xlsx = xgb.XGBRegressor()
xgb_model_xlsx.fit(X_train_xlsx, y_train_xlsx)

# PREDICT AND EVALUATE
y_pred_csv = xgb_model_csv.predict(X_test_csv)
mse_csv = np.mean((y_test_csv - y_pred_csv) ** 2)
print("Mean Squared Error (CSV):", mse_csv)

y_pred_xlsx = xgb_model_xlsx.predict(X_test_xlsx)
mse_xlsx = np.mean((y_test_xlsx - y_pred_xlsx) ** 2)
print("Mean Squared Error (XLSX):", mse_xlsx)


# PREDICT TOP 10 COINS WITH HIGHEST PRICES (CSV)
if 'Currency' in df_csv.columns:
    predicted_prices_csv = xgb_model_csv.predict(X_csv)
    df_csv['Predicted Price'] = predicted_prices_csv
    top_10_coins_csv = df_csv.nlargest(10, 'Predicted Price')
    print(Fore.GREEN + "Top 10 Coins (CSV):")
    for index, row in top_10_coins_csv.iterrows():
        print(f"Row {index}: Currency - {row['Currency']}, Price - {row['Predicted Price']}")
else:
    print("Column 'Currency' not found in the dataframe.")

# PREDICT TOP 10 COINS WITH HIGHEST PRICES (XLSX)
predicted_prices_xlsx = xgb_model_xlsx.predict(X_xlsx)
df_xlsx['Predicted Price'] = predicted_prices_xlsx
top_10_coins_xlsx = df_xlsx.nlargest(10, 'Predicted Price')
print(Fore.GREEN + "Top 10 Coins (XLSX):")
for index, row in top_10_coins_xlsx.iterrows():
    name = label_encoder.inverse_transform([int(row['Name'])])[0]
    print(f"Row {index}: Name - {name}, Price - {row['Predicted Price']}")

Mean Squared Error (CSV): 7.42085722665552e-08
Mean Squared Error (XLSX): 0.00022000770948981413
Top 10 Coins (CSV):
Row 86919: Currency - blackcoin, Price - 0.9995359182357788
Row 173812: Currency - diamond, Price - 0.9995359182357788
Row 279374: Currency - golem-network-tokens, Price - 0.9995359182357788
Row 435703: Currency - parkbyte, Price - 0.9995359182357788
Row 575369: Currency - terracoin, Price - 0.9995359182357788
Row 594913: Currency - trust, Price - 0.9995359182357788
Row 602475: Currency - unbreakablecoin, Price - 0.9995359182357788
Row 130000: Currency - civic, Price - 0.9995293021202087
Row 133249: Currency - clubcoin, Price - 0.9995293021202087
Row 357567: Currency - lykke, Price - 0.9995293021202087
Top 10 Coins (XLSX):
Row 371: Name - LockChain, Price - 0.9989762902259827
Row 724: Name - PlatinumBAR, Price - 0.9762399196624756
Row 135: Name - Santiment Net..., Price - 0.9657490253448486
Row 112: Name - PayPie, Price - 0.961028516292572
Row 168: Name - WaBi, Price - 0

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# FUNCTION TO HANDLE OUTLIERS
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# FUNCTION TO CALCULATE VOLATILITY BANDS
def calculate_volatility_bands(df, column, window=20, factor=2):
    rolling_std = df[column].rolling(window=window).std()
    upper_band = df[column] + factor * rolling_std
    lower_band = df[column] - factor * rolling_std
    return upper_band, lower_band

# FUNCTION TO PREPARE DATA FOR RNN
def prepare_rnn_data(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

# DATASET
df_csv = pd.read_csv(r"C:\Users\nh013\Desktop\Crypto fluctuation of 3 consecutive years\Cryptocurrency Prices by Date.csv")
df_xlsx = pd.read_excel(r"C:\Users\nh013\Desktop\Crypto fluctuation of 3 consecutive years\All Currencies Table.xlsx")

# SELECT COLUMNS
df_csv = df_csv[['Currency', 'Price']]
df_xlsx = df_xlsx[['Name', 'Price']]

# IDENTIFY MISSING VALUES
df_csv.dropna(inplace=True)
df_xlsx.dropna(inplace=True)

# CONVERT 'Price' COLUMN TO NUMERIC
df_csv['Price'] = pd.to_numeric(df_csv['Price'], errors='coerce')
df_xlsx['Price'] = pd.to_numeric(df_xlsx['Price'], errors='coerce')

# DROP ROWS WITH MISSING VALUES
df_csv.dropna(subset=['Price'], inplace=True)
df_xlsx.dropna(subset=['Price'], inplace=True)

# HANDLE OUTLIERS
df_csv = handle_outliers(df_csv, 'Price')
df_xlsx = handle_outliers(df_xlsx, 'Price')

# NORMALIZE AND SCALE
scaler_csv = MinMaxScaler()
df_csv[['Price']] = scaler_csv.fit_transform(df_csv[['Price']])

scaler_xlsx = MinMaxScaler()
df_xlsx[['Price']] = scaler_xlsx.fit_transform(df_xlsx[['Price']])

# CONVERT TO SEQUENTIAL DATA FOR RNN
window_size = 20  # Number of previous time steps to consider
X_csv, y_csv = prepare_rnn_data(df_csv['Price'].values, window_size)
X_xlsx, y_xlsx = prepare_rnn_data(df_xlsx['Price'].values, window_size)

# BUILD RNN MODEL
model_csv = Sequential()
model_csv.add(LSTM(64, input_shape=(window_size, 1)))
model_csv.add(Dense(1))
model_csv.compile(optimizer='adam', loss='mean_squared_error')
model_csv.fit(X_csv, y_csv, epochs=10, batch_size=32)

model_xlsx = Sequential()
model_xlsx.add(LSTM(64, input_shape=(window_size, 1)))
model_xlsx.add(Dense(1))
model_xlsx.compile(optimizer='adam', loss='mean_squared_error')
model_xlsx.fit(X_xlsx, y_xlsx, epochs=10, batch_size=32)

# PREDICT FUTURE PRICES
future_dates = pd.date_range(start='2024-01-01', periods=10, freq='D')

X_csv_pred = df_csv['Price'].values[-window_size:].reshape(1, window_size, 1)
predicted_prices_csv = []
for _ in range(10):
    predicted_price_csv = model_csv.predict(X_csv_pred)[0][0]
    predicted_prices_csv.append(predicted_price_csv)
    X_csv_pred = np.append(X_csv_pred[:, 1:, :], [[[predicted_price_csv]]], axis=1)

X_xlsx_pred = df_xlsx['Price'].values[-window_size:].reshape(1, window_size, 1)
predicted_prices_xlsx = []
for _ in range(10):
    predicted_price_xlsx = model_xlsx.predict(X_xlsx_pred)[0][0]
    predicted_prices_xlsx.append(predicted_price_xlsx)
    X_xlsx_pred = np.append(X_xlsx_pred[:, 1:, :], [[[predicted_price_xlsx]]], axis=1)

# INVERSE TRANSFORM PREDICTED PRICES
predicted_prices_csv = scaler_csv.inverse_transform(np.array(predicted_prices_csv).reshape(-1, 1))
predicted_prices_xlsx = scaler_xlsx.inverse_transform(np.array(predicted_prices_xlsx).reshape(-1, 1))

# OUTPUT PREDICTED PRICES
for date, price_csv, price_xlsx, currency, name in zip(future_dates, predicted_prices_csv, predicted_prices_xlsx, df_csv['Currency'], df_xlsx['Name']):
    print(f"Date: {date.strftime('%Y-%m-%d')} | CSV Currency: {currency} | CSV Price: {price_csv[0]:.2f} | XLSX Name: {name} | XLSX Price: {price_xlsx[0]:.2f}")


Epoch 1/10
16815/16815 [==============================] - 189s 11ms/step - loss: 0.0020
Epoch 2/10
16815/16815 [==============================] - 175s 10ms/step - loss: 0.0019
Epoch 3/10
16815/16815 [==============================] - 186s 11ms/step - loss: 0.0019
Epoch 4/10
16815/16815 [==============================] - 192s 11ms/step - loss: 0.0019
Epoch 5/10
16815/16815 [==============================] - 187s 11ms/step - loss: 0.0019
Epoch 6/10
16815/16815 [==============================] - 235s 14ms/step - loss: 0.0019
Epoch 7/10
16815/16815 [==============================] - 199s 12ms/step - loss: 0.0019
Epoch 8/10
16815/16815 [==============================] - 200s 12ms/step - loss: 0.0019
Epoch 9/10
16815/16815 [==============================] - 183s 11ms/step - loss: 0.0019
Epoch 10/10
16815/16815 [==============================] - 182s 11ms/step - loss: 0.0019
Epoch 1/10
39/39 [==============================] - 4s 10ms/step - loss: 0.0401
Epoch 2/10
39/39 [=====================